# ABS Quarterly National Accounts 5206

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Four-quarter-rolling-totals---original-series" data-toc-modified-id="Four-quarter-rolling-totals---original-series-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Four quarter rolling totals - original series</a></span></li><li><span><a href="#Growth-charts---based-on-seasonally-adjusted-series" data-toc-modified-id="Growth-charts---based-on-seasonally-adjusted-series-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Growth charts - based on seasonally adjusted series</a></span></li><li><span><a href="#Gross-Value-Added-(GVA)-by-Industry" data-toc-modified-id="Gross-Value-Added-(GVA)-by-Industry-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Gross Value Added (GVA) by Industry</a></span></li><li><span><a href="#Household-Final-Consumption-Expenditure" data-toc-modified-id="Household-Final-Consumption-Expenditure-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Household Final Consumption Expenditure</a></span></li><li><span><a href="#Taxes" data-toc-modified-id="Taxes-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Taxes</a></span></li><li><span><a href="#Government-Benefits-Payments" data-toc-modified-id="Government-Benefits-Payments-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Government Benefits Payments</a></span></li><li><span><a href="#COVID-HFCE-comparison" data-toc-modified-id="COVID-HFCE-comparison-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>COVID HFCE comparison</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys
import platform
import psutil
from pathlib import Path
import calendar
import re

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.units as munits

# abs specific imports
from abs_common import (get_ABS_meta_and_data,
                        get_identifier,
                        finalise_plot,
                        plot_Qgrowth)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
# cache directory
CACHE_DIR = '../cache'
Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

# charts directory
CHART_DIR = '../charts/5206'
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)

## Python environment

In [3]:
N = 50
print('-'*N)
print(f"System:         {platform.system()}")
print(f"Release:        {platform.release()}")
#print(f"Version:        {platform.version()}")
print(f"Machine:        {platform.machine()}")
print(f"Processor:      {platform.processor()}")
print(f"RAM:            {round(psutil.virtual_memory().total / 1024**3)}GB")
print('-'*N)
print(f"Python:         {platform.python_version()}")
print(f"Psutil:         {psutil.__version__}")
print(f"Pandas:         {pd.__version__}")
print(f"Numpy:          {np.__version__}")
print(f"Matplotlib:     {mpl.__version__}")
print('-'*N)

--------------------------------------------------
System:         Darwin
Release:        20.1.0
Machine:        x86_64
Processor:      i386
RAM:            64GB
--------------------------------------------------
Python:         3.8.3
Psutil:         5.7.0
Pandas:         1.0.5
Numpy:          1.18.5
Matplotlib:     3.2.2
--------------------------------------------------


## Get data from ABS

In [4]:
url_template = ('https://www.abs.gov.au/statistics/economy/national-accounts/'
                'australian-national-accounts-national-income-expenditure-and-product/'
                'MONTH-YEAR/All_time_series_workbooks.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

File has been cached already
File for sep-2020 of size 5.4 MB


## Plot

### Set-up some constants we will use across all plots

In [5]:
RECENT = data.index.max() - pd.DateOffset(years=3)
plot_times = (None, RECENT)
plot_frames = (data.copy(), data[data.index >= RECENT].copy())
plot_tags = ('full', 'recent')

headline_series = {
    # series: table,
    'Gross domestic product: Chain volume measures ;': '1',
    'GDP per capita: Chain volume measures ;': '1',
    'Gross value added market sector: Chain volume measures ;': '1',
    'Net domestic product: Chain volume measures ;': '1',
    'Real gross domestic income: Chain volume measures ;': '1',
    'Real gross national income: Chain volume measures ;': '1',
    'Real net national disposable income: Chain volume measures ;': '1',
}

CVM = 'Chain volume measures'

### Four quarter rolling totals - original series

In [6]:
series_type = "Original"
for description, table in headline_series.items():

    series, units = get_identifier(meta, description,
                               series_type, table)
    rolling4Q = data[series].rolling(4).sum()
    units = f'{units} / Year'
    
    for plot_time, plot_tag in zip(plot_times, plot_tags):
        plot_data = rolling4Q
        if plot_time:
            plot_data = rolling4Q[rolling4Q.index >= plot_time]
        ax = plot_data.plot(lw=2, color="#dd0000")
        
        title = description.replace(' ;', '')
        
        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'Source: ABS 5206; table {table}',
            lfooter=f'4Q rolling sum {series_type.lower()} series'
        )

### Growth charts - based on seasonally adjusted series

In [7]:
series_type = "Seasonally Adjusted"
for description, table in headline_series.items():
    
    series, _ = get_identifier(meta, description,
                               series_type, table)
        
    for plot_from, plot_tag in zip(plot_times, plot_tags):

        title = description.replace(' ;', '')
        
        plot_Qgrowth(data[series].copy(), 
            title, plot_from, 
            plot_tag, CHART_DIR, 
            rfooter=f'Source: ABS 5206, table {table}',
            lfooter=f'From the {series_type.lower()} series')

### Gross Value Added (GVA) by Industry

In [8]:
# let's focus on major industries
table = '6'
series_type = 'Seasonally Adjusted'
industries = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) &
    (meta['Unit'] == '$ Millions') # limit to money
]['Data Item Description']
industry_start = pd.Timestamp('1974-09-01')

for industry in industries:

    series, units = get_identifier(meta, industry,
                               series_type, table)
    units = f'{units}/Qtr'
    
    for plot_frame, plot_tag in zip(plot_frames, plot_tags):
        used_frame = plot_frame[plot_frame.index >= industry_start]
        ax = used_frame[series].plot(lw=2, color="#dd0000")

        title = industry
        if not title.endswith(") ;"):
            title = re.sub("^[^\(]+\(", "(", title)
        title = f' GVA: {title}'

        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'Source: ABS 5206; table {table}',
            lfooter=(f'{series_type.capitalize()}, '
                     f'{CVM.lower()}')
        )

###  Household Final Consumption Expenditure

In [9]:
table = '8'
series_type = 'Seasonally Adjusted'
hfce = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) &
    (meta['Unit'] == '$ Millions') & # limit to money
    (meta['Data Item Description'].str.contains(CVM))
]['Data Item Description']

for description in hfce:
    series, units = get_identifier(meta, description,
                                   series_type, table)
    units = f'{units}/Qtr'
    
    for plot_frame, plot_tag in zip(plot_frames, plot_tags):
        ax = plot_frame[series].plot(lw=2, color="#dd0000")
        
        title = description.replace(f': {CVM} ;', '')
        title = f'Household spending: {title}'
        
        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'Source: ABS 5206; table {table}',
            lfooter=(f'{series_type.capitalize()}, '
                     f'{CVM.lower()}')
        )

### Taxes

In [10]:
table = '22'
series_type = 'Seasonally Adjusted'
tax_plots = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) 
]['Data Item Description']

for plot in tax_plots:
    
    # data capture
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f'{units}/Qtr'
    series = data[ident]
    series = series[series.index >= RECENT]
    if series.isna().all():
        # don't plot empty series
        continue
    
    # plot
    ax = series.plot(lw=2, color="#dd0000")
    title = plot.replace(' ;', '')
    finalise_plot(ax, title, f'{units} (Current Prices)', 
                  'govt-tax', CHART_DIR,
                  rfooter=f'Source: ABS 5206, table {table}',
                  lfooter=f'{series_type.capitalize()} series')

### Government Benefits Payments

In [11]:
table = '23'
series_type = 'Original'
plots_payments = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) 
]['Data Item Description']

for plot in plots_payments:
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f'{units}/Qtr'
    series = data[ident]
    series = series[series.index >= RECENT]
    if series.isna().all():
        # don't plot empty series
        continue

    # plot
    ax = series.plot(lw=2, color="#dd0000")
    
    # - from ABS description to plot title
    title = plot.replace('General government - National ;  ',
                         'Federal Govt - ')
    title = title.replace('General government - State and local ;  ',
                         'State or Local Govt - ')
    title = title.replace('General government ;  ',
                         'All Govt - ')
    title = title.replace(' ;', '')
    
    finalise_plot(ax, title, f'{units} (Current Prices)', 
                  'govt-spending', CHART_DIR,
                  rfooter=f'Source: ABS 5206, table {table}',
                  lfooter=f'{series_type.capitalize()} series')
        

### COVID HFCE comparison

In [12]:
# used for all COVID indexes
reference = pd.Timestamp('2019-12-01')
ref_string = f'{calendar.month_abbr[reference.month]} {reference.year}'
index_base = 100
df = data[data.index >= RECENT]
series_type = "Seasonally Adjusted"

In [13]:
# HFCE component of state final demand 
# Victoria compared with the rest of Australia
states = {
    # state: table
    'NSW': '26',
    'Vic': '27',
    'Qld': '28',
    'SA':  '29',
    'WA':  '30',
    'Tas': '31',
    'NT':  '32',
    'ACT': '33',
}

HFCE = 'Households ;  Final consumption expenditure - '
CVM = 'Chain volume measures'
plots = meta.loc[
    meta['Data Item Description'].str.contains(HFCE)
    & meta['Data Item Description'].str.contains(CVM)
    & ~meta['Data Item Description'].str.contains('Revisions')
    & (meta['Table'] == states['Vic'])
    & (meta['Series Type'] == series_type)
]['Data Item Description']

for plot in plots:
    rest = []
    for state in states:
        series, units = get_identifier(meta, plot,
                                       series_type, 
                                       states[state])
        if state == 'Vic':
            victoria = df[series]
        else: 
            rest.append(series)
    rest = df[rest].sum(axis=1)
    plotable = pd.DataFrame([victoria, rest]).T
    plotable.columns = ['Victoria', 'Rest of Australia']
    reference_row = plotable.loc[reference]
    plotable = plotable.div(reference_row) * index_base
    title = plot.replace(HFCE, '')
    title = title.replace(f': {CVM} ;', '')
    title = f'Household spending: {title}'

    ax = plotable.plot(lw=2, color=['darkorange', 'blue'])
    finalise_plot(ax, title, 
                  f'Index ({ref_string} = {index_base})', 
                  'COVID', CHART_DIR,
                  rfooter=f'Source: ABS 5206; tables {",".join(states.values())}',
                  lfooter=(f'{series_type.capitalize()}, '
                           f'{CVM.lower()}')
                 )


## Finished

In [14]:
print('Finished')

Finished
